In [1]:
# use condaenv_basic.yml to do this

import duckdb



In memory

In [2]:
duckdb.execute("CREATE TABLE tbl AS SELECT 42 a")
df = duckdb.execute("SELECT * FROM tbl").df()
print(df)

    a
0  42


Store as a persistant file

In [3]:
# create persisted database and write data
with duckdb.connect(database="my_duckdb.db") as write_conn:
    write_conn.execute("CREATE TABLE tbl AS SELECT 42 a")

In [4]:
# somewhere else: read and process data
with duckdb.connect(database="my_duckdb.db", read_only=True) as read_conn:
    df = read_conn.execute("SELECT * FROM tbl").df()
    print(df)

    a
0  42


In [5]:
import pandas as pd

In [8]:
def calculate_monthly_taxi_stats_duckdb(conn: duckdb.DuckDBPyConnection, path: str) -> pd.DataFrame:
    return (
        conn.sql(f"""
            select 
                period,
                count(*) AS num_rides,
                round(avg(trip_duration), 2) AS avg_trip_duration,
                round(avg(trip_distance), 2) AS avg_trip_distance,
                round(sum(trip_distance), 2) as total_trip_distance,
                round(avg(total_amount), 2) as avg_trip_price,
                round(sum(total_amount), 2) as total_trip_price,
                round(avg(tip_amount), 2) as avg_tip_amount
            from (
                select
                    date_part('year', tpep_pickup_datetime) as trip_year,
                    strftime(tpep_pickup_datetime, '%Y-%m') as period,
                    epoch(tpep_dropoff_datetime - tpep_pickup_datetime) as trip_duration,
                    trip_distance,
                    total_amount,
                    tip_amount
                from parquet_scan("{path}")
                where trip_year >= 2021 and trip_year <= 2024
            )
            group by period
            order by period
        """).df()
    )


# Run
res_duckdb = calculate_monthly_taxi_stats_duckdb(conn=duckdb, path=r"C:\Users\TristramArmour\OneDrive - Innovisk\Documents\data\nytaxis\*parquet")

In [7]:
res_duckdb

,period,num_rides,avg_trip_duration,avg_trip_distance,total_trip_distance,avg_trip_price,total_trip_price,avg_tip_amount
0,2021-01,1369753,834.52,4.63,6344692.58,17.47,2.393565e+07,1.92
1,2021-02,1371689,893.13,4.28,5873685.40,17.61,2.415231e+07,1.91
2,2021-03,1925130,883.86,4.38,8440408.16,17.92,3.449731e+07,1.94
3,2021-04,2171215,930.56,7.39,16035976.30,18.56,4.029785e+07,2.06
4,2021-05,2507075,954.40,8.42,21116924.95,18.89,4.735560e+07,2.20
5,2021-06,2834204,1009.59,6.93,19651902.42,19.41,5.501904e+07,2.32
6,2021-07,2822113,997.47,7.79,21972025.90,19.88,5.609682e+07,2.36
7,2021-08,2788300,994.34,7.67,21385365.68,20.20,5.632110e+07,2.39
8,2021-09,2963778,1050.01,6.58,19512409.58,20.87,6.185309e+07,2.50
9,2021-10,3463485,1045.21,7.11,24634431.07,20.28,7.023040e+07,2.53
